**Group-1**

**Title:** Classification of Doremon cartoon characters from a custom Image dataset using a CNN model.

**Team members:**
Santosh Reddy Edulapalle - A20501739
Venkata Siva Rupesh Akurati - A20501754
Jack Harrison Mohr -A20503445

**This is our .ipynb file for the project. We are going to code everything in this file.**

**Image dataset:** We are using OpenCV to read the video and capture image with rate of 1 frame per second.

**ImageSS** This function takes in path of the video and uses OpenCV to read each frame and captures the images.

In [21]:
import os

#importing cv2 from OpenCV
import cv2
from datetime import datetime

def ImageSS(videoPath):

    videoObject = cv2.VideoCapture(videoPath)
    frameNumber = 3000

    # checks whether frames were extracted
    boolean = 1
    start = datetime.now()
    try:
        while boolean:
           # cv2.waitKey(10) {doesnt reduce fps}
            boolean, image = videoObject.read()
            cv2.imwrite("frame%d.jpg" %frameNumber, image)
            frameNumber += 1
    #exception handling
    except:
        print("All the frames are read!")
    finally:
        end = datetime.now()
    #calculating execution time.
    executionTime = (end - start).total_seconds() * 10**3
    print(f"The execution time of above program is : {executionTime:.03f}ms")


The below cell is commented to prevent it from accidental running.Uncomment it to use the program to capture video frames.

In [22]:
#ImageSS("/Users/santosh/Desktop/doremon6.mp4")

**generate_neg_description_txt_file** This function runs over the negative images folder creates a new text tile with all the names in negative image folder + negative tag attached to it.

In [23]:
#Creating our own cascade classifier from scratch.
# we are going to train the classifier with our own doremon data.

def generate_neg_description_txt_file():
    with open('negative.txt','w') as f:
        for imageName in os.listdir('negative'):
            f.write('negative/' + imageName + '\n')
generate_neg_description_txt_file()

negatives.txt is relatively easy to do. But for positives.txt, we need to manually upload the frame of the face for each image. To help with it, openCV has annotations app which is only available on version 3x.
 We installed openCV 3.4.16 to use its annotations functions. For the rest of the project, we will be using openCV latest version.
Having had so many troubles with openCV 3.4.16 on my mac, and spending one full day on debugging, I decided to move on to a windows PC and finish the annotation process. I will be transferring the trained model, related files ( annotations.txt) to this mac for further project build.


In [24]:
#changes yet to make.
import numpy as np

# settings
videopath = '/Users/santosh/Desktop/dorVid/'
exclusion = {
    '.DS_Store'
}

# gets the import location for videos (dataset).
def get_dataset(videopath=videopath):
    videos = os.listdir(os.path.dirname(videopath))
    for item in videos:
        if item in exclusion:
            videos.remove(item)
    return videos

# detect character by using a custom trained haar cascade for each character.
def detect(character, video, show_video=True):
    cap = cv2.VideoCapture(videopath + video)
    Dor_face_cascade = cv2.CascadeClassifier('cascadeAll.xml')
    results_path = os.path.join('results/' + "Dor_char")

    # make a folder in results for our recognised faces.
    if not os.path.exists(results_path) and character['save'] == True:
        os.mkdir(results_path)

    while(1):
        # grab a frame.
        ret, frame = cap.read()
        faces = None


        if character['name'] == "Doremon":
            # detect faces in our image.
            # try only with frame, no extra parameters.
            faces = Dor_face_cascade.detectMultiScale(frame,
                                                  scaleFactor=1.05,
                                                  minNeighbors=50,
                                                  minSize=(24, 24),
                                                  flags=cv2.CASCADE_SCALE_IMAGE
                                                  )
        elif character['name'] == "Nobita":
            faces = Dor_face_cascade.detectMultiScale(frame,
                                                  scaleFactor=1.05,
                                                  minNeighbors=50,
                                                  minSize=(24, 24),
                                                  flags=cv2.CASCADE_SCALE_IMAGE
                                                  )
        elif character['name'] == "Shizuka":
            faces = Dor_face_cascade.detectMultiScale(frame,
                                                      scaleFactor=1.05,
                                                      minNeighbors=50,
                                                      minSize=(24, 24),
                                                      flags=cv2.CASCADE_SCALE_IMAGE
                                                      )

        # loop over detected faces.
        for (x, y, w, h) in faces:
            # setup region of interest (ROI) for the captured face.
            roi = frame[y:y+h, x:x+w]

            frame_number = str(int(cap.get(cv2.CAP_PROP_POS_FRAMES)))

            # write detected face to disk.
            if character['save'] == True:
                cv2.imwrite(results_path + '/' + "character" + '_frame_' + frame_number + '.png', roi)

            if show_video is True:
                # display detection box for visual purposes.
                cv2.rectangle(frame, (x, y), (x+w, y+h), character['detect_color'], 2)
                cv2.putText(frame, character['name'], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)
            else:
                print('detected face @ frame ' + frame_number)

        if show_video is True:
            # display our image.
            try:
                cv2.imshow('frame', frame)
            except:
                break

            # quit or (next video) on esc.
            esc = cv2.waitKey(30) & 0xff
            if esc == 27:
                break

    # destroy & release resources.
    cv2.destroyAllWindows()
    cap.release()

# process all our videos.
def process(character):
    videos = get_dataset()
    print('number of videos: ' + str(len(videos)))

    for video in enumerate(videos):
        episode = video[1]

        # dump frames and save to disk each character.
        print('attempting to detect ' + character['name'] +  ' in \'' + episode + '\'')

        # detect our character.
        detect(character, video[1], show_video=True)

def main():
    # step 1: prepare our results folder.
    if not os.path.exists('results'):
        os.mkdir('results')

    # step 2: process all our videos to detect Tom & Jerry.
    characters = [
        {
            'name':      "Doremon",
            'detect_color': (26,225,248),
            'save':		 True,
            'cascade':   'doremon.xml'
        },
        {
            'name':      "Nobita",
            'detect_color': (236,195, 80),
            'save':		 True,
            'cascade':   'nobita.xml'
        },
        {
            'name':      "Shizuka",
            'detect_color': (255,182,193),
            'save':		 True,
            'cascade':   'shizuka.xml'
        }
    ]
    # process characters...
    [process(character) for character in characters]
    print('done')

if __name__ == '__main__':
    main()

number of videos: 1
attempting to detect Doremon in 'Doremon1.mp4'
number of videos: 1
attempting to detect Nobita in 'Doremon1.mp4'
number of videos: 1
attempting to detect Shizuka in 'Doremon1.mp4'
done
